In [76]:
from twython import TwythonStreamer
import csv
import json
import re
from textblob import TextBlob 
import pandas as pd
from pandas import DataFrame

In [77]:
def clean_tweet(tweet): 
         
        #Utility function to clean tweet text by removing links, special characters 
        # using simple regex statements. 
        
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

In [80]:
def get_tweet_sentiment(tweet): 
    #Utility function to classify sentiment of passed tweet 
    # using textblob's sentiment method 
        
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [81]:
# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = clean_tweet(tweet['text'])
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    d['date']= tweet['created_at']
    #d['sentiment'] = get_tweet_sentiment(clean_tweet['text']))
    d['sentiment'] = get_tweet_sentiment(tweet['text'])
    return d

In [82]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [83]:
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [84]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream

with open(r'saved_tweets.csv', 'a', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['hashtags','text', 'user', 'user_loc', 'date', 'sentiment'])

topic = "BTC"

stream.statuses.filter(track='BTC')

In [3]:
import pandas as pd
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv")
tweets.head()

,hashtags,text,user,user_loc,date,sentiment
0,[],Great moments in crypto,rod3000,Sydney,Tue Sep 17 01:36:31 +0000 2019,positive
1,NaN,NaN,NaN,NaN,NaN,NaN
2,[],RT Fancy a slice of 10 000 in Bitcoin We re le...,Annabelbeyerfr1,NaN,Tue Sep 17 01:36:33 +0000 2019,positive
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"['brand', 'tech', 'technology', 'innovation', ...",RT on SALE brand tech technology innovation bi...,SantchiWeb,Wirral,Tue Sep 17 01:36:34 +0000 2019,neutral


In [5]:
tweets.head()

,hashtags,text,user,user_loc,date,sentiment
0,[],Great moments in crypto,rod3000,Sydney,Tue Sep 17 01:36:31 +0000 2019,positive
1,NaN,NaN,NaN,NaN,NaN,NaN
2,[],RT Fancy a slice of 10 000 in Bitcoin We re le...,Annabelbeyerfr1,NaN,Tue Sep 17 01:36:33 +0000 2019,positive
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"['brand', 'tech', 'technology', 'innovation', ...",RT on SALE brand tech technology innovation bi...,SantchiWeb,Wirral,Tue Sep 17 01:36:34 +0000 2019,neutral


In [6]:
tweets2 = tweets.dropna()
tweets2.head()

,hashtags,text,user,user_loc,date,sentiment
0,[],Great moments in crypto,rod3000,Sydney,Tue Sep 17 01:36:31 +0000 2019,positive
4,"['brand', 'tech', 'technology', 'innovation', ...",RT on SALE brand tech technology innovation bi...,SantchiWeb,Wirral,Tue Sep 17 01:36:34 +0000 2019,neutral
6,[],RT A testing bug on Kraken let users buy and s...,Cryptofever5,South East Asia,Tue Sep 17 01:36:41 +0000 2019,positive
8,[],I ll sell my Bitcoin when Neveruary 1 st comes...,ClaudiwNeamtu,Denmark,Tue Sep 17 01:36:43 +0000 2019,neutral
10,[],Bitcoin s 30 Day Volatility Just Reached Its L...,CharlesLBovaird,Boston MA,Tue Sep 17 01:36:48 +0000 2019,neutral


In [59]:
print (len(list(tweets2['text'])))

765


In [102]:
fnametweets = "TWEETS2.csv"
tweets2.to_csv(fnametweets, encoding='utf-8', index=False)

In [103]:
c = tweets2.groupby('sentiment').count()
c
list(c["hashtags"])

[73, 411, 281]

In [112]:
data1 = {}
data1['negative'] = [list(c["hashtags"])[0]]
data1['neutral'] = [list(c["hashtags"])[1]]
data1['positive'] = [list(c["hashtags"])[2]]
data1

{'negative': [73], 'neutral': [411], 'positive': [281]}

In [113]:
df2 = pd.DataFrame(data1)
df2

,negative,neutral,positive
0,73,411,281


In [114]:
#To save cleaned location in CSV file
df2.to_csv("sentimentval.csv", encoding='utf-8', index=False)

In [104]:
data = {'sentiment value': list(c["hashtags"])}
df1 = DataFrame(data, columns= ['negative','neutral', 'positive'])
print (df1)

Empty DataFrame
Columns: [negative, neutral, positive]
Index: []


In [69]:
file_sentiment1 = 'sentiment1.csv'
with open(file_sentiment1, 'a') as file:
    writer = csv.writer(file)
    writer.writerow(['negative','neutral', 'positive'])
    writer.writerow(list(c["hashtags"]))


In [ ]:
df1 = pd.read_csv(file_sentiment1)
df1

In [85]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim()

C:\Users\asi40\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


In [86]:
import numbers
import decimal

In [87]:
file_location = 'tweets_location.csv'
with open(file_location, 'a') as file:
    writer = csv.writer(file)
    writer.writerow(['location','latitude', 'longitude'])
    

In [88]:
# from geopy import geocoders  
# gn = geocoders.GeoNames(username='cmoreno2020')
# gn.geocode("Cleveland, OH", exactly_one=False)[0]


In [89]:

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'location': [], 'latitude': [], 'longitude': []}


for count, user_loc in enumerate(tweets2.user_loc):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        
        if isinstance(location.latitude, numbers.Number):
            #print(user_loc)
            coordinates['location'].append(user_loc)
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
    except:
        pass
    
    


Sydney
Wirral
South East Asia
Denmark
Boston MA
Blockchain
California, USA
New Delhi, India
New York, NY
Global
San Francisco
Panama
Sandusky, OH
Los Angeles, CA
BCH
United States
United States
Japan
blockchain
0x
Los Angeles, CA
United States
PORTUGUESA - GUANARE 
Dallas, TX
Canada
nigeria
New Delhi, India
Valencia, España
Cape Cod
United States
Montréal, Québec
where
Chicago, IL
Rio de Janeiro, Brazil
Los Angeles, CA
Los Angeles
Pakistan
🕊️
Everywhere
Los Angeles, CA
California, USA
New Delhi, India
Nairobi, Kenya
Australia
SF
North Pole, AK
United States
California, USA
On Chain
Pakistan
Matter Doesn't Matter
Home
Madhya Pradesh, India
West Lawn, Chicago
Pakistan
where
Malaysia
Georgia, USA
S. Bridge
California, USA
MARS
España
The World
Appleton, WI
Central Mississippi
Tokyo-to, Japan
Simi Valley, CA
Washington, DC
Around the World
S. Bridge
New York, USA
Georgia, USA
Lagos, Nigeria
Nairobi, Kenya
Bandungan ,Semarang Indonesia
Georgia, USA
Harrisburg, PA
temanggung
Pune
Halifax, No

Nigeria
Los Angeles, CA
Berlin, Germany
Lagos, Nigeria
Kingman, AZ
DC
Texas, USA
Worldwide
London, England
California, USA
Hong Kong
Maharashtra, India
San Francisco, CA
Maharashtra, India
Denver, CO
Japan
New York, USA
Myrtle Beach, SC USA 
Chicago
South Carolina, USA
Home
Hunts Point Market, Bronx
Miami, FL
Guarenas
United States
New York, New York
Australia
EMEA
Minneapolis, MN
Planet Earth 
Miami
euro
Cairo 
London, England
 Alabama
上海, 中华人民共和国
Lagos, Nigeria
San Diego
San Diego
Water
Mississauga, Ontario
Philippines
California, USA
Bakersfield, CA


In [90]:
len(coordinates['location'])

657

In [91]:
for i in range(0, len(coordinates['location'])):
     with open(file_location, 'a', encoding='utf-8') as file:
         writer = csv.writer(file)
         writer.writerow([coordinates['location'][i], coordinates['latitude'][i], coordinates['longitude'][i]])


In [96]:
user_location = pd.read_csv(file_location)
user_location1 = user_location.dropna()
len(list(user_location1["latitude"]))

657

In [95]:
#To save cleaned location in CSV file
user_location1.to_csv("twitter_loc.csv", encoding='utf-8', index=False)

In [94]:
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
#gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)
gmap.heatmap(list(user_location1['latitude']), list(user_location1['longitude']), radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")